In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 드라이버 실행 및 첫 페이지 로딩
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = 'https://terms.naver.com/list.naver?cid=42157&categoryId=42157'
driver.get(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "desc")))

# 전체 페이지 크롤링 함수
def all_pages():
    page_number = 1
    while True:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "desc")))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        results = []
        for li in soup.select('ul.content_list > li'):
            term_tag = li.select_one('strong.title > a')
            desc_tag = li.select_one('p.desc')
            if term_tag and desc_tag:
                term = term_tag.get_text(strip=True)
                desc = desc_tag.get_text(strip=True)
                results.append({'용어': term, '설명': desc})

        yield results

        # 다음 페이지로 이동
        next_page_num = page_number + 1
        next_link = soup.select_one(f'div#paginate a[href*="page={next_page_num}"]')
        if next_link and 'href' in next_link.attrs:
            next_url = "https://terms.naver.com" + next_link['href']
            driver.get(next_url)
            page_number = next_page_num
        else:
            break

# 크롤링 실행 및 CSV 저장
all_data = []
for page_data in all_pages():
    all_data.extend(page_data)

df = pd.DataFrame(all_data)
df.to_csv('군사용어사전.csv', index=False, encoding='utf-8-sig')

print(f"총 {len(df)}개의 용어가 '군사용어사전.csv' 파일에 저장되었습니다.")

총 6902개의 용어가 '군사용어사전.csv' 파일에 저장되었습니다.
